In [ ]:
#Importing libraries
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
import numpy as np
from sklearn.model_selection import train_test_split
#from sklearn.utils.multiclass import unique_labels
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
#from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
#from keras.datasets import cifar10
from os import listdir
from PIL import Image as PImage
from PIL import ImageOps
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import os
import glob
from IPython.display import Image
from zipfile import ZipFile
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with ZipFile('/content/drive/MyDrive/Research/singlePCBerror_4combined.zip', 'r') as f:

  f.extractall('/content/drive/MyDrive/singlePCBerror_4combined')

In [ ]:
folder_path=r'/content/drive/MyDrive/singlePCBerror_4combined/singlePCBerror_4combined'
imagesList = listdir(folder_path)

In [ ]:
len(imagesList)

In [ ]:
os.chdir("/content/drive/MyDrive/singlePCBerror_4combined/singlePCBerror_4combined/")
images = []
for filename in glob.glob('*.jpg'): 
    im=PImage.open(filename)
    img=ImageOps.grayscale(im)
    img=img.resize((224,224))
    img = np.array(img).astype('float32')
    images.append(img)

In [ ]:
len(images)

In [ ]:
len(imagesList)

In [ ]:
imagesList[0]

In [ ]:
csv_data=pd.read_csv("/content/drive/MyDrive/Research/updated_singlePCBerror_4combined_labels.csv",index_col=False)

In [ ]:
final_list=[]
len_list=len(imagesList)
for i in range(0,len_list):
    datarow=csv_data[csv_data['Image_Name']==imagesList[i]]
    #print(datarow[['Image_Name','Classes']])
    df_reset=datarow.set_index('Classes')
    index=df_reset.index
    for a in index:
        classes=a
        data=[imagesList[i],classes]
    #print(index[0])
    #abc_list=df_reset['Classes']
    #print(abc_list)
    #classes=df_reset.iloc[:,:]
    #print(classes)
    print(data)
    final_list.append(data)    
df = pd.DataFrame(final_list,columns =['Image_Name','Classes'])
df.to_csv(r'/content/drive/MyDrive/Research/final_updated_singlePCBerror_4combined_labels.csv', index = False)

In [ ]:
#from google.colab import files

#uploaded = files.upload()
labels=pd.read_csv("/content/drive/MyDrive/Research/final_updated_singlePCBerror_4combined_labels.csv")
#labels=pd.read_csv("/home/sshaji/updated_singlePCBerror_4combined_labels.csv")
imageName=labels.pop('Image_Name')
labels = labels.pop('Classes')
new_labels = np.array([i-1 for i in labels.values])

In [ ]:
images=np.array(images)
print(imagesList[0])
#plt.imshow(images[0])
#plt.show()

In [ ]:
Image(filename=imagesList[0]) 

In [ ]:
#Splitting into train and test data
train_images, x_test, train_labels, y_test = train_test_split(images, new_labels, test_size=0.2, random_state=42)

#splitting the train data(images and labels)to train and validation data
#x_val - validation images
#y_val - validation labels
x_train,x_val,y_train,y_val=train_test_split(train_images,train_labels,test_size=.3)

In [ ]:
print((np.array(x_train).shape,np.array(y_train).shape))
print((np.array(x_val).shape,np.array(y_val).shape))
print((np.array(x_test).shape,np.array(y_test).shape))

In [ ]:
s1 = pd.Series(y_train)
y_train=pd.get_dummies(s1)
print(y_train.shape)

In [ ]:
s2 = pd.Series(y_val)
y_val=pd.get_dummies(s2)
print(y_val.shape)

In [ ]:
s3 = pd.Series(y_test)
y_test=pd.get_dummies(s3)
print(y_test.shape)

In [ ]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

In [ ]:
x_train=np.array(x_train).reshape(44907, 224, 224, 1)
x_val=np.array(x_val).reshape(19246, 224, 224,1)
x_test=np.array(x_test).reshape(16039, 224, 224,1)
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

In [ ]:
# Converting the images from grayscale to RGB for input to the VGG16 network
x_train=tf.image.grayscale_to_rgb(tf.convert_to_tensor(x_train), name=None)
x_val=tf.image.grayscale_to_rgb(tf.convert_to_tensor(x_val), name=None)
x_test=tf.image.grayscale_to_rgb(tf.convert_to_tensor(x_test), name=None)

In [ ]:
# Printing the shape of train, test and validation images
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

In [ ]:
#Fitting the augmentation to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [ ]:
# Get the convolutional base of a VGG16 network pretrained on ImageNet dataset
vgg16_convoltional_base = VGG16(weights='imagenet', include_top=False)
inputlayer=vgg16_convoltional_base.get_layer(index=0)
print(inputlayer)
inputlayer.trainable=False
vgg16_convoltional_base.summary()

# Provide the shape of the input data here - 224x224x3
input = Input(shape=(224, 224, 3))
vgg16_output = vgg16_convoltional_base(input)

# Flatten the output, add FC layers, dropout and output layer
final_output = Flatten(name='flatten')(vgg16_output)
final_output = Dense(512, activation='relu', name='fc1',kernel_regularizer=l2(0.01))(final_output)
final_output = Dense(512, activation='relu', name='fc2',kernel_regularizer=l2(0.01))(final_output)
final_output = Dropout(0.3)(final_output)
final_output = Dense(6, activation='softmax', name='predictions')(final_output)

#Create the model and print the model schematic
my_model = Model(inputs=input, outputs=final_output)
my_model.summary()

In [ ]:
# Compiles the model for training

my_model.compile(optimizer=Adam(learning_rate=0.00001), 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
#Defining the hyperparameters
batch_size= 40
epochs=30
#learn_rate=.001

In [ ]:
#Training the model 
history = my_model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = epochs, steps_per_epoch = len(x_train)//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = len(x_test)//batch_size, verbose=1)

In [ ]:
#Testing the model
test_loss, test_acc = my_model.evaluate(x_test,  y_test)

In [ ]:
#Plotting the graphs for accuracy and validation_accuracy
plt.plot(history.history['accuracy'],"-b", label="accuracy")
plt.plot(history.history['val_accuracy'],"-r", label="val_accuracy")
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
#Plotting the graphs for loss and validation_loss
plt.plot(history.history['loss'],"-b", label="loss")
plt.plot(history.history['val_loss'],"-r", label="val_loss")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
#Defining the labels
class_names = ['open', 'short', 'mousebite', 'spur', 'copper',
               'pin-hole']

In [ ]:
# making predictions
prediction=my_model.predict(x_test)
for i in range(25):
  pil_img = tf.keras.preprocessing.image.array_to_img(x_test[i])
  plt.imshow(pil_img)
  plt.title("Prediction:"+class_names[np.argmax(prediction[i])])
  plt.show()